In [ ]:
!pip install langchain langchain-community langchain-core pypdf faiss-cpu sentence-transformers langchain_google_genai langchain_openai langchain_chroma

# 모델 불러오기

In [ ]:
import os
from google.colab import userdata # 코랩 Secrets 기능 사용을 위한 임포트
from langchain_google_genai import ChatGoogleGenerativeAI
# 코랩 Secrets에서 GOOGLE_API_KEY 가져오기
try:
    google_api_key = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = google_api_key
    print("GOOGLE_API_KEY가 환경 변수로 설정되었습니다.")
except userdata.SecretNotFoundError:
    print("오류: 'GOOGLE_API_KEY' 비밀을 코랩 Secrets에 설정해주세요. ")
    exit() # API 키가 없으면 스크립트 종료
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
print(f"Selected LLM: {llm.__class__.__name__}")

GOOGLE_API_KEY가 환경 변수로 설정되었습니다.
Selected LLM: ChatGoogleGenerativeAI


# RunnablePassthrough

*   입력을 변경하지 않거나 추가 키를 더하여 전달



In [ ]:
from langchain_core.prompts import PromptTemplate


# prompt, llm
prompt = PromptTemplate.from_template("{num} 의 10배는?")
# chain 생성
chain = prompt | llm

In [ ]:
# chain 실행
chain.invoke({"num": 5})

AIMessage(content='5의 10배는 50입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--10d7f197-eee4-4813-9cc5-cfe2f16bd1c7-0', usage_metadata={'input_tokens': 8, 'output_tokens': 13, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

In [ ]:
# chain 실행_변수가 하나라면 딕셔너리 형태가 아니어도 됨
chain.invoke(5)

AIMessage(content='5의 10배는 50입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--00e28b57-6077-4751-9b2a-c77d17773817-0', usage_metadata={'input_tokens': 8, 'output_tokens': 13, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# runnable_그대로 출력
RunnablePassthrough().invoke({"num": 10})

{'num': 10}

In [ ]:
runnable_chain = {"num": RunnablePassthrough()} | prompt | llm

# dict 값이 RunnablePassthrough() 로 변경
runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--990a3eb6-bd1a-489d-9612-1a30f7c48904-0', usage_metadata={'input_tokens': 9, 'output_tokens': 15, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [ ]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

#RunnableParallel
*  여러 runnable 병렬로 처리

In [ ]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성. 여러 Runnable 인스턴스를 병렬로 실행
runnable = RunnableParallel(
    # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달. 입력된 데이터를 그대로 통과시킴
    passed=RunnablePassthrough(),
    # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용, 'mult' 람다 함수를 할당.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키워드 인자로 람다 함수를 전달.
    modified=lambda x: x["num"] + 1,
)

# runnable 인스턴스에 {'num': 1} 딕셔너리를 입력으로 전달하여 invoke 메소드 호출
runnable.invoke({"num": 1})


{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

## 체인

In [ ]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | llm
)


In [ ]:
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'capital': AIMessage(content='대한민국의 수도는 **서울**입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--11ffc4f5-2229-4aaf-9a46-555eec9d8ebd-0', usage_metadata={'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}}),
 'area': AIMessage(content='대한민국의 면적은 약 **10만 847,600 제곱킬로미터**입니다.  하지만 이 숫자는 섬 지역을 포함한 전체 면적이며,  육지 면적만을 따지면 약 99,720 제곱킬로미터입니다.  어떤 면적을 기준으로 하느냐에 따라 약간의 차이가 있을 수 있습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--1876739d-a426-4837-ad98-6206438e55f9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 103, 'total_tokens': 112, 'input_token_details': {'cache_read': 0}})}

# RunnableLambda
*  사용자 정의 함수 매핑

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from datetime import datetime


def get_today(a):
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(None)

'May-30'

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = llm

# chain 을 생성합니다.
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 출력
print(chain.invoke(3))

5월 30일 생일인 유명인 3명은 다음과 같습니다.

* **콜린 파렐(1976년 5월 30일)**: 아일랜드 배우
* **앤젤라 바셋(1958년 8월 16일)**:  5월 30일이 생일이 아니므로 제외했습니다.
* **제이슨 스태덤(1967년 7월 26일)**: 5월 30일이 생일이 아니므로 제외했습니다.


죄송합니다. 5월 30일 생일인 유명인을 세 명 찾기가 어렵습니다.  더 많은 정보를 제공해 드릴 수 없어 유감입니다.  다른 날짜를 시도해 보시겠습니까? 또는 특정 직업이나 국적을 지정해 주시면 도움이 될 수 있습니다.


In [ ]:
# itemgetter
from operator import itemgetter

data = [
    {"name": "Alice", "age": 30, "city": "New York"},
    {"name": "Bob", "age": 24, "city": "London"},
    {"name": "Charlie", "age": 35, "city": "Paris"},
]

# 'name' 키의 값을 가져오는 함수 생성
get_name = itemgetter('name')
print(get_name(data[0]))
print([get_name(person) for person in data])

Alice
['Alice', 'Bob', 'Charlie']


In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


# 문장의 길이를 반환
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
chain1 = prompt | llm

chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)

In [ ]:
chain.invoke({"word1": "hello", "word2": "world"})


AIMessage(content='5 + 25 = 30 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash-latest', 'safety_ratings': []}, id='run--9b0e1038-f5c2-4e14-811d-03c36f9496d7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})